# b. 使用 Llama-cpp-python 加载量化后的 LLM 大模型（GGUF）

> 引导文章：[19b. 从加载到对话：使用 Llama-cpp-python 本地运行量化 LLM 大模型（GGUF）](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/19b.%20从加载到对话：使用%20Llama-cpp-python%20本地运行量化%20LLM%20大模型（GGUF）.md)

代码文件没有显卡要求，在个人计算机上均可进行对话。

**模型文件约为 4 GB**。

这里还有一个简单的 [🎡 AI Chat 脚本](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/chat.py)供你尝试，详见：[CodePlayground](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/README.md#当前的玩具)，点击 `►` 或对应的文本展开。

Transformers 关于 GPTQ & AWQ 加载的相关链接：[文章 19a](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/19a.%20从加载到对话：使用%20Transformers%20本地运行量化%20LLM%20大模型（GPTQ%20%26%20AWQ）.md) | [代码文件 16a](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/16a.%20使用%20Transformers%20加载量化后的%20LLM%20大模型（GPTQ%20%26%20AWQ）.ipynb)

在线链接：[Kaggle - b](https://www.kaggle.com/code/aidemos/16b-llama-cpp-python-llm-gguf) | [Colab - b](https://colab.research.google.com/drive/1AhgC0qDaqWBXAI9eSbwTStGgvgFfLOpf?usp=sharing)

## Llama-cpp-python 

### 环境配置

为了确保之后的 "offload" 正常工作，需要进行一些额外的工作。

首先，找到 CUDA 的安装路径：

In [ ]:
!find /usr/local -name "cuda" -exec readlink -f {} \;

复制对应（最短）的路径，修改 `cuda_home` 环境变量：

In [ ]:
import subprocess

# 设置环境变量
cuda_home = "/usr/local/cuda-12.1"  # 请替换为你的路径

command = f"""
CMAKE_ARGS="-DGGML_CUDA=on \
            -DCUDA_PATH={cuda_home} \
            -DCUDAToolkit_ROOT={cuda_home} \
            -DCUDAToolkit_INCLUDE_DIR={cuda_home}/include \
            -DCUDAToolkit_LIBRARY_DIR={cuda_home}/lib64 \
            -DCMAKE_CUDA_COMPILER={cuda_home}/bin/nvcc" \
FORCE_CMAKE=1 \
pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir --verbose
"""

subprocess.run(command, shell=True, check=True)

### GGUF

#### 安装库

In [ ]:
!pip install gguf

#### 导入库

In [ ]:
from llama_cpp import Llama

下面介绍两种导入模型的方法，实际执行时本地/自动导入二选一。

#### 本地导入模型


In [ ]:
# 如果你已经配置过了，可以直接在 Notebook 中执行下面的命令下载。
!export HF_ENDPOINT=https://hf-mirror.com
!./hfd.sh bartowski/Mistral-7B-Instruct-v0.3-GGUF --include Mistral-7B-Instruct-v0.3-Q4_K_M.gguf --tool aria2c -x 16

根据模型路径导入模型，注意，文件位于 `<repo_id>` 文件夹下，以当前下载的文件为例：

In [ ]:
# 指定本地模型的路径
model_path = "./Mistral-7B-Instruct-v0.3-GGUF/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf"

# 加载模型
llm = Llama(
    model_path=model_path,
    # n_gpu_layers=-1,  # 取消注释使用 GPU 加速
    # verbose=False,  # 取消注释减少模型方面的信息打印
)

#### 自动下载并导入


In [ ]:
# 指定仓库的名称和文件名
repo_id = "bartowski/Mistral-7B-Instruct-v0.3-GGUF"
filename = "Mistral-7B-Instruct-v0.3-Q4_K_M.gguf"
#filename = "*Q4_K_M.gguf"  # 使用通配符也是可以的


# 下载并加载模型
llm = Llama.from_pretrained(
    repo_id=repo_id,
    filename=filename,
    #n_gpu_layers=-1,  # 取消注释使用 GPU 加速
    #verbose=False,  # 取消注释减少模型方面的信息打印
)

#### 推理测试

简单使用以下命令进行推理测试：

In [ ]:
# 输入文本
input_text = "Hello, World!"

# 生成输出
output = llm(input_text, max_tokens=50)

# 打印生成的文本
print(output['choices'][0]['text'])

### 流式输出


In [ ]:
prompt = "人工智能的未来发展方向是什么？"

output = llm.create_chat_completion(
    messages=[{
        "role": "user",
        "content": prompt
    }],
    max_tokens=200,
    stream=True
)

for chunk in output:
    delta = chunk['choices'][0]['delta']
    if 'role' in delta:
        print(delta['role'], end=': ', flush=True)
    elif 'content' in delta:
        print(delta['content'], end='', flush=True)

查看 output 的构造：

In [ ]:
from itertools import islice

prompt = "人工智能的未来发展方向是什么？"

output = llm.create_chat_completion(
    messages=[{
        "role": "user",
        "content": prompt
    }],
    max_tokens=200,
    stream=True
)

print(type(output))

# 将生成器转换为列表
output_list = list(islice(output, 3))

# 获取前 3 个条目
output_list[:3]

将刚刚对于流式输出的处理抽象为函数便于后续调用：

In [ ]:
def handle_stream_output(output):
    """
    处理流式输出，将生成的内容逐步打印出来。
    
    参数：
        output: 生成器对象，来自 create_chat_completion 的流式输出
    """
    for chunk in output:
        delta = chunk['choices'][0]['delta']
        if 'role' in delta:
            print(f"{delta['role']}: ", end='', flush=True)
        elif 'content' in delta:
            print(delta['content'], end='', flush=True)
            
# 使用示例
prompt = "人工智能的未来发展方向是什么？"

output = llm.create_chat_completion(
    messages=[{
        "role": "user",
        "content": prompt
    }],
    max_tokens=200,
    stream=True
)

handle_stream_output(output)

### 多轮对话

让我们自定义一个交互的对话类（需要注意到 handle_stream_output() 有所修改）。


In [ ]:
from llama_cpp import Llama

def handle_stream_output(output):
    """
    处理流式输出，将生成的内容逐步打印出来，并收集完整的回复。

    参数：
        output: 生成器对象，来自 create_chat_completion 的流式输出

    返回：
        response: 完整的回复文本
    """
    response = ""
    for chunk in output:
        delta = chunk['choices'][0]['delta']
        if 'role' in delta:
            print(f"{delta['role']}: ", end='', flush=True)
        elif 'content' in delta:
            content = delta['content']
            print(content, end='', flush=True)
            response += content
    return response

class ChatSession:
    def __init__(self, llm):
        self.llm = llm
        self.messages = []

    def add_message(self, role, content):
        """
        添加一条消息到会话中。

        参数：
            role: 消息角色，通常为 'user' 或 'assistant'
            content: 消息内容
        """
        self.messages.append({"role": role, "content": content})

    def get_response_stream(self, user_input):
        """
        获取模型对用户输入的响应（流式输出）。

        参数：
            user_input: 用户输入的文本

        返回：
            response: 完整的回复文本
        """
        self.add_message("user", user_input)
        
        try:
            output = self.llm.create_chat_completion(
                messages=self.messages,
                stream=True  # 开启流式输出
            )
            
            response = handle_stream_output(output)  # 同时打印和收集回复
            
            self.add_message("assistant", response.strip())
            # print(len(response),len(response.strip()))
            return response.strip()
        except Exception as e:
            print(f"\n发生错误: {e}")
            return ""

# 初始化模型（假设使用本地路径）
model_path = "./Mistral-7B-Instruct-v0.3-GGUF/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf"
llm = Llama(
    model_path=model_path,
    n_gpu_layers=-1,  # 根据需要卸载到 GPU
    n_ctx=4096,       # 设置上下文窗口大小，尝试注释这行进行多轮对话，看看会发生什么
    verbose=False,    # 禁用详细日志输出
)

# 创建会话实例
chat = ChatSession(llm)
        
# 开始对话
while True:
    prompt = input("User: ")
    # 退出对话条件（当然，你也可以直接终止代码块）
    if prompt.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break
    chat.get_response_stream(prompt)
    print()  # 换行以便下一次输入，这是因为之前的 print 都设置了 end=''

In [ ]:
# 使用 bye 退出上面的对话后打印查看
print(chat.messages)